In [1]:
########## SETUP: SEE BELOW ##########


import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from google.colab import drive

########## MODIFY THIS: #######
memberpath = '/content/drive/My Drive/wbdb2.csv'    #### Change to wherever you want the initial stats site pull to be saved
wbdbpath = '/content/drive/My Drive/apidb.csv'      #### Change to wherever you want the DB to be saved
drive.mount('/content/drive', force_remount=True)                     #### Comment this line out (#) if you are not using Google Colab
########################################


Mounted at /content/drive


In [2]:
# setup, we define the URL and parse bs4's request response
base_url = 'https://stats.warbrokers.io/squads'
page = requests.get(base_url)
soup = BeautifulSoup(page.text, 'html.parser')

In [3]:
#searches for the squad list on the main page

squad_list = soup.find('ul', {'id': 'squadCollapsible'})
squad_names = [a.text for a in squad_list.find_all('a', class_='squadLink')]


In [4]:
#defines dataframe to hold values
player_data = pd.DataFrame(columns=['Squad', 'PlayerName', 'ID'])

In [5]:
#loops through and pulls data from squad pages
for squad_name in tqdm(squad_names):
    squad_url = f'https://stats.warbrokers.io/squads/{squad_name}'
    squad_page = requests.get(squad_url)
    squad_soup = BeautifulSoup(squad_page.text, 'html.parser')

    player_headers = squad_soup.find_all('div', class_='squad-player-header')

    for player_header in player_headers:
        player_link = player_header.find('a')
        player_name = player_link.text
        player_id = player_link['href'].split('/')[-1]

        player_data = pd.concat([player_data, pd.DataFrame({'Squad': squad_name, 'PlayerName': player_name, 'ID': player_id}, index=[0])], ignore_index=True)
        player_data.to_csv(memberpath)

100%|██████████| 266/266 [23:47<00:00,  5.37s/it]


In [6]:
#defines dataframe for API pull
apidb = pd.DataFrame(columns=['Squad', 'PlayerName', 'UID', 'Level', 'XP', 'KillsElo', 'GamesElo', 'Jumps', 'Losses', 'ZombieDeaths', 'ZombieKills', 'ZombieWins', 'DamageDealt', 'Banned', 'Steam', 'Time', 'JoinTime', 'PingTime', 'TimeAliveCount', 'TimeAliveLongest', 'TimeAlive', 'ZombieTimeAliveCount', 'ZombieTimeAlive'])

playerdata_iter = tqdm(player_data.iterrows(), total=len(player_data), desc="Fetching Player Data")

Fetching Player Data:   0%|          | 0/949 [00:00<?, ?it/s]

In [ ]:
#loops through initial player list dataframe and augments with info pulled via API. Saves to .csv on every iteration for debugging.
for index, row in playerdata_iter:
    uid = row['ID']
    squad = row['Squad']
    player_name = row['PlayerName']

    # API URL for the user
    api_url = f'https://wbp-wbapi.web.app/players/getPlayer?uid={uid}'

    try:
        response = requests.get(api_url)
        player_info = response.json()

        # Extract relevant information from the API response
        level = player_info.get('level', 0)
        xp = player_info.get('xp', 0)
        kills_elo = player_info.get('killsELO', 0)
        games_elo = player_info.get('gamesELO', 0)
        coins = player_info.get('coins', 0)
        jumps = player_info.get('number_of_jumps', 0)
        losses = sum(player_info.get('losses', {}).values())
        zombie_deaths = player_info.get('zombie_deaths', 0)
        zombie_kills = player_info.get('zombie_kills', 0)
        zombie_wins = player_info.get('zombie_wins', 0)
        damage_dealt = sum(player_info.get('damage_dealt', {}).values())
        banned = player_info.get('banned', False)
        steam = player_info.get('steam', False)
        time = player_info.get('time', 0)
        join_time = player_info.get('joinTime', 0)
        ping_time = player_info.get('ping_time', 0)
        time_alive_count = player_info.get('time_alive_count', 0)
        time_alive_longest = player_info.get('time_alive_longest', 0)
        time_alive = player_info.get('time_alive', 0)
        zombie_time_alive_count = player_info.get('zombie_time_alive_count', 0)
        zombie_time_alive = player_info.get('zombie_time_alive', 0)

        # Create a DataFrame for the current player
        player_df = pd.DataFrame({'Squad': squad, 'PlayerName': player_name, 'UID': uid, 'Level': level, 'XP': xp, 'KillsElo': kills_elo, 'GamesElo': games_elo, 'Coins': coins, 'Jumps': jumps, 'Losses': losses, 'ZombieDeaths': zombie_deaths, 'ZombieKills': zombie_kills, 'ZombieWins': zombie_wins, 'DamageDealt': damage_dealt, 'Banned': banned, 'Steam': steam, 'Time': time, 'JoinTime': join_time, 'PingTime': ping_time, 'TimeAliveCount': time_alive_count, 'TimeAliveLongest': time_alive_longest, 'TimeAlive': time_alive, 'ZombieTimeAliveCount': zombie_time_alive_count, 'ZombieTimeAlive': zombie_time_alive}, index=[0])

        # Concatenate the player DataFrame to apidb
        apidb = pd.concat([apidb, player_df], ignore_index=True)
        apidb.to_csv(wbdbpath)

    except Exception as e:
        print(f"Error retrieving data for {player_name}: {str(e)}")

    apidb['Steam'] = apidb['Steam'].astype(bool) #set explicitly as bool

Fetching Player Data:   0%|          | 1/949 [14:31<229:26:58, 871.33s/it]<ipython-input-9-798d200c600f>:41: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  apidb = pd.concat([apidb, player_df], ignore_index=True)
Fetching Player Data:   0%|          | 2/949 [14:31<94:25:12, 358.94s/it] <ipython-input-9-798d200c600f>:41: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  apidb = pd.concat([apidb, player_df], ignore_index=True)
Fetching Player Data:   0%|          | 3/949 [14:31<51:17:03, 195.16s/it]<ipython-input-9-798d200c600f>:41: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  apidb = pd.concat([apidb, player_df